In [1]:
def get_library_path()->str:

    cwd = os.getcwd()
    lst = cwd.split(os.sep)[:-1]

    path_lib = f'{os.sep}'

    for elem in lst:
        path_lib = os.path.join(path_lib, elem)

    return path_lib

# imports

import os
import sys

path_to_lib = get_library_path()

sys.path.append(path_to_lib)
import luxgiant_clinical.TwoCatAnalysis as two
from luxgiant_clinical.Helpers import categories_recoder

import pandas as pd
import numpy as np

In [2]:
# load data

folder_path = os.path.join(path_to_lib, 'data/source')

df = pd.read_csv(
    os.path.join(folder_path, 'cleaned_file.csv'), 
    low_memory=False
)

In [3]:
# filter data to keep only patients with disease duration information

mask_patients= (df['Status']=='Patient')
mask_length  = (~df['pdsl'].isnull()) 

df_cases = df[mask_patients & mask_length].reset_index(drop=True)
del df

df_cases["pdsl"] = df_cases["pdsl"].map({"<=5":'PD duration <=5 yrs', ">5":'PD duration >5 yrs'})

df_cases.shape

(7473, 716)

In [4]:
# rename and select first group of variables

variables_dict = {
    "initial_motor_symptom_s___1"   : "Onset with motor symptoms",
    "initial_motor_symptom_s___2"   : "Rest tremor as onset symptom" ,
    "initial_motor_symptom_s___12"  : "Bradykinesia as onset symptom" ,
    "initial_motor_symptom_s___13"  : "Rigidity as onset symptom",
    "initial_motor_symptom_s___3"   : "Postural/action tremor at onset",
    "initial_motor_symptom_s___8"   : "Reduced dexterity at onset",
    "initial_motor_symptom_s___9"   : "Microphagia at onset",
    "initial_motor_symptom_s___11"  : "Reduced arm swing at onset",
    "initial_motor_symptom_s___16"  : "Gait disorder at onset",
    "initial_motor_symptom_s___14"  : "Freezing at onset",
    "initial_motor_symptom_s___15"  : "Falls at onset",
    "initial_non_motor_symptom___1" : "Non Motor Symp at onset",
    "initial_non_motor_symptom___18": "Constipation at onset",
    "initial_non_motor_symptom___8" : "Insomnia as onset symptom",
    "initial_non_motor_symptom___4" : "RBD at onset",
    "initial_non_motor_symptom___4" : "RLS at onset",
    "initial_non_motor_symptom___20": "Urinary urgency as onset symptom",
    "initial_non_motor_symptom___13": "Hyperhidrosis as onset symptom",
    "initial_non_motor_symptom___10": "Anosmia/hyposmia as onset symptom",
    "initial_non_motor_symptom___2" : "Cognitive symptoms at onset",
    "current_motor_symptom_s___2"   : "Current rest tremor",
    "current_motor_symptom_s___3"   : "Current postural/action tremor",
    "current_motor_symptom_s___14"  : "Current bradykinesia",
    "current_motor_symptom_s___4"   : "Current rigidity",
    "current_motor_symptom_s___15"  : "Current Gait disorder",
    "current_motor_symptom_s___16"  : "Current Falls",
    "current_motor_symptom_s___17"  : "Current Freezing",
    "current_motor_symptom_s___18"  : "Current Dyskinesias",
    "current_motor_symptom_s___19"  : "Current Motor fluctuation",
    "current_non_motor_symptom___3" : "Current NMS Psychosis",
    "current_non_motor_symptom___5" : "Current depression",
    "current_non_motor_symptom___6" : "Current RBD",
    "current_non_motor_symptom___7" : "Current RLS",
    "current_non_motor_symptom___9" : "Current anosmia/hyposmia",
    "current_non_motor_symptom___2" : "Current cognitive impairment",
    "current_non_motor_symptom___10": "Current EDSS",
    "current_non_motor_symptom___11": "Current insomnia",
    "current_non_motor_symptom___14": "Current orthostasis",
    "current_non_motor_symptom___16": "Current hyperhidrosis",
    "current_non_motor_symptom___17": "Current seborrhea",
    "current_non_motor_symptom___20": "Current constipation",
    "current_non_motor_symptom___22": "Current urinary incontinence"
}
variables = ['participant_id', 'pdsl'] + list(variables_dict.keys())

# statistical measures
stats_meas = {
    'n'     : ['Onset with motor symptoms', 'Rest tremor as onset symptom', 'Bradykinesia as onset symptom', 
               'Rigidity as onset symptom', 'Postural/action tremor at onset', 'Reduced dexterity at onset', 
               'Microphagia at onset', 'Reduced arm swing at onset', 'Gait disorder at onset', 'Freezing at onset', 
               'Falls at onset', 'Non Motor Symp at onset', 'Constipation at onset', 'Insomnia as onset symptom', 
               'RLS at onset', 'Urinary urgency as onset symptom', 'Hyperhidrosis as onset symptom', 
               'Anosmia/hyposmia as onset symptom', 'Cognitive symptoms at onset', 'Current rest tremor', 
               'Current postural/action tremor', 'Current bradykinesia', 'Current rigidity', 'Current Gait disorder', 
               'Current Falls', 'Current Freezing', 'Current Dyskinesias', 'Current Motor fluctuation', 
               'Current NMS Psychosis', 'Current depression', 'Current RBD', 'Current RLS', 
               'Current anosmia/hyposmia', 'Current cognitive impairment', 'Current EDSS', 'Current insomnia', 
               'Current orthostasis', 'Current hyperhidrosis', 'Current seborrhea', 'Current constipation', 
               'Current urinary incontinence']
    }

groups = ['PD duration <=5 yrs', 'PD duration >5 yrs']

In [5]:
# keep only variables for analysis

df_cases_1 = df_cases[variables].copy()
df_cases_1 = df_cases_1.rename(columns=variables_dict)

del df_cases

In [6]:
# recode variable values
motor_non_motor = ["Onset with motor symptoms", "Non Motor Symp at onset"]
remainig = [feat for feat in stats_meas['n'] if feat not in motor_non_motor]

df_cases_1 = categories_recoder(df_cases_1, motor_non_motor, {"Checked":0, "Unchecked":1})
df_cases_1 = categories_recoder(df_cases_1, remainig, {"Checked":1, "Unchecked":0})

In [7]:
df_10 = two.report_proportion(
    data_df    =df_cases_1,
    variables  =stats_meas['n'],
    groups     =groups,
    grouping_by='pdsl',
)
df_10 = two.final_formatter(df_10, groups)

In [8]:
# save result in csv file

df_10.to_csv(os.path.join(path_to_lib, 'data/auxiliar/aux_table_10.csv'), index=False)
df_10

,Variable,Statistical Measure,PD duration <=5 yrs,PD duration >5 yrs,p-value,Total,Available Samples for Analysis
0,Onset with motor symptoms,n (%),4126 (99.5),3317 (99.7),0.1091,7443 (99.6),7473
1,Rest tremor as onset symptom,n (%),2686 (64.8),1967 (59.1),p<0.001,4653 (62.3),7473
2,Bradykinesia as onset symptom,n (%),1709 (41.2),1285 (38.6),0.0239,2994 (40.1),7473
3,Rigidity as onset symptom,n (%),1273 (30.7),1067 (32.1),0.1999,2340 (31.3),7473
4,Postural/action tremor at onset,n (%),1076 (25.9),839 (25.2),0.478,1915 (25.6),7473
5,Reduced dexterity at onset,n (%),449 (10.8),449 (13.5),p<0.001,898 (12.0),7473
6,Microphagia at onset,n (%),224 (5.4),245 (7.4),p<0.001,469 (6.3),7473
7,Reduced arm swing at onset,n (%),848 (20.4),725 (21.8),0.155,1573 (21.0),7473
8,Gait disorder at onset,n (%),571 (13.8),534 (16.1),0.0057,1105 (14.8),7473
9,Freezing at onset,n (%),234 (5.6),306 (9.2),p<0.001,540 (7.2),7473
